In [150]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
import pandas as pd
from nltk.corpus import stopwords
import nltk
import numpy as np
from nltk.tokenize import word_tokenize
from keras.optimizers import Adam
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
stop_words = stopwords.words('indonesian')

In [151]:
df_train = pd.read_csv('datasets/new_dataset.csv')
MAX_SEQUENCE_LENGTH = 250
selected_column = df_train.columns[1:]
df_test = pd.read_csv('datasets/data_test.csv')
total_train_data = len(df_train)
df_train = df_train.append(df_test)

/tmp/ipykernel_56360/893036123.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df_test)


In [152]:
def preprocessingText(text: str):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    text = text.lower()
    text = stemmer.stem(text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text_tokenize = word_tokenize(text)
    text = [t for t in text_tokenize if t not in stop_words]
    text = " ".join(text)
    return text

In [153]:
df_train['questions'] = df_train['questions'].apply(lambda x: preprocessingText(x))

In [154]:
num_words = 1000
max_words = 100000

In [155]:
X = df_train['questions'].values
tokenizer = tf.keras.preprocessing.text.Tokenizer(max_words)
tokenizer.fit_on_texts(X)
X_train = tokenizer.texts_to_sequences(X)
X_train = tf.keras.utils.pad_sequences(X_train, MAX_SEQUENCE_LENGTH)

In [156]:
X_train_data = X_train[0:total_train_data]
X_test_data = X_train[total_train_data:]

In [157]:
columns = df_train.columns
y_train = np.array(df_train[columns[1:]])
y_train_data = y_train[0:total_train_data]
y_test_data = y_train[total_train_data:]

In [158]:
embeddings_index = {}
with open('w2vec_wiki_id300_0.txt', encoding='utf-8') as file:
    for line in file:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        embed = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embed

embed_size = 300
word_index = tokenizer.word_index
embed_num_words = min(max_words, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embed_size), dtype='float32')

KeyboardInterrupt: 

In [ ]:
for word, i in word_index.items():
    if i >= max_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
MAX_LEN = 250
input = tf.keras.layers.Input(shape=(MAX_LEN,))
x = tf.keras.layers.Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1], weights=[embedding_matrix], trainable=False)(input)
x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, return_sequences=True, dropout=0.1,
                                                      recurrent_dropout=0.1))(x)
x = tf.keras.layers.Conv1D(128, kernel_size=3, padding="valid", kernel_initializer="glorot_uniform")(x)
avg_pool = tf.keras.layers.GlobalAveragePooling1D()(x)
max_pool = tf.keras.layers.GlobalMaxPooling1D()(x)

x = tf.keras.layers.concatenate([avg_pool, max_pool])

preds = tf.keras.layers.Dense(y_train.shape[1], activation="sigmoid")(x)

model = tf.keras.Model(input, preds)


optimizer = Adam(learning_rate=1e-3)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    cp_callback
]
# model.fit(X_train, y_train, verbose=1, epochs=10, validation_split=0.2, callbacks=callbacks)
model.fit(X_train_data, y_train_data, verbose=1, epochs=50, validation_data=(X_test_data, y_test_data), callbacks=callbacks)

Epoch 1/50
21/21 [==============================] - ETA: 0s - loss: 0.3723 - accuracy: 0.0583
Epoch 1: saving model to training_1/cp.ckpt
21/21 [==============================] - 26s 1s/step - loss: 0.3723 - accuracy: 0.0583 - val_loss: 0.1944 - val_accuracy: 0.0493
Epoch 2/50
21/21 [==============================] - ETA: 0s - loss: 0.1871 - accuracy: 0.1166
Epoch 2: saving model to training_1/cp.ckpt
21/21 [==============================] - 23s 1s/step - loss: 0.1871 - accuracy: 0.1166 - val_loss: 0.1812 - val_accuracy: 0.0493
Epoch 3/50
21/21 [==============================] - ETA: 0s - loss: 0.1710 - accuracy: 0.2255
Epoch 3: saving model to training_1/cp.ckpt
21/21 [==============================] - 23s 1s/step - loss: 0.1710 - accuracy: 0.2255 - val_loss: 0.1675 - val_accuracy: 0.1761
Epoch 4/50
21/21 [==============================] - ETA: 0s - loss: 0.1516 - accuracy: 0.3252
Epoch 4: saving model to training_1/cp.ckpt
21/21 [==============================] - 23s 1s/step - loss: 

In [ ]:
model.evaluate(X_train, y_train)

25/25 [==============================] - 4s 165ms/step - loss: 0.0409 - accuracy: 0.7166


[0.040889158844947815, 0.7166246771812439]

In [ ]:
'{:.3f}'.format(3.6592403e-13)
#  9.9997795e-01 -> 0.99997795

'0.000'

In [ ]:
model.save('tf_model')

INFO:tensorflow:Assets written to: tf_model/assets


INFO:tensorflow:Assets written to: tf_model/assets


In [ ]:
model = tf.keras.models.load_model('tf_model')

In [159]:
faq = 'cara connect vpn'
faq = preprocessingText(faq)
print(faq)
tokenize_faq = word_tokenize(faq)
tokenize_faq = [val for val in tokenize_faq if val not in stop_words]
faq = " ".join(tokenize_faq)
new_faq = [faq]
seq = tokenizer.texts_to_sequences(new_faq)
padded = tf.keras.utils.pad_sequences(seq, MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)
# print(pred)
threshold = 0.5
found = False
for i in range(len(selected_column)):
    if pred[0][i] > threshold:
        found = True
        pred1 = '{:.3f}'.format(pred[0][i])
        print(selected_column[i], pred1)

if not found:
    print("Tags tidak ditemukan")

connect vpn
1/1 [==============================] - 0s 95ms/step
Tags tidak ditemukan
